<a href="https://colab.research.google.com/github/leehyell/AIStudy24/blob/master/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지금까지 배운 알고리즘은 훈련세트와 테스트 세트를 나누어 평가를 진행했다.
* 테스트 세트로 평가를 하면 테스트 세트에 맞는 모델이 만들어지는 결론이 나온다.
    * 테스트 세트에 '**일반화 성능**'을 올바르게 예측하려면 가능한 한 **테스트 세트를 사용하지 말아야** 한다.
    * 모델을 만들고 나서 마지막에 딱 한 번 사용하는 것이 좋다.
<br/><br/>

결정트리에서 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝을 해보자.

###검증 세트
* 테스트 세트를 사용하지 않으려면 모델이 과대 적합인지, 과소 적합인지 판단하기 어렵다.
* 테스트 세트를 사용하지 않고 측정하는 간단한 방법은 훈련 세트를 또 나누는 것이다.
    * 훈련 세트 60%, 테스트 세트 20%, 검증 세트 20%
    * └ 실무에서 많이 쓰는 기법.
* 테스트 하고 싶은 매개변수를 바꿔가면서 가장 좋은 모델이 나올 수 있음.
* 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련함.
* 마지막 테스트에서 최종 점수를 평가함.
    * 1. 훈련 세트에서 모델을 훈련.
    * 2. 검증 세트로 모델을 평가.
    * 3. 테스트 세트를 이용하여 최종 점수를 평가함.

In [64]:
import pandas as pd

#원래 원본 데이터 - 6,497개
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
#훈련용, 테스트용 나누기
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
#훈련용에서 검증용 다시 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)
#원래 5197개였던 훈련세트가 4,157개로 줄고, 검증 세트가 1,040개가 됨.

(4157, 3) (1040, 3)


In [2]:
#sub_input, sub_target, val_input, val_target을 사용해 모델을 만들고 평가함.
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

#↓ 0.9971133028626413
print(dt.score(sub_input, sub_target))
#↓ 0.864423076923077
print(dt.score(val_input, val_target))
#이미 sub_input이 99.7%라 과대 적합.
#매개변수를 바꿔서 더 좋은 모델을 찾아야 함.

0.9971133028626413
0.864423076923077


In [3]:
#검증 세트를 생성하느라 훈련 세트가 줄어듦.
    #이럴 때 많은 데이터를 훈련하면 결과가 좋아짐.
#교차 검증: 검증 세트를 떼어내어 평가하는 과정을 여러번 반복함.
#훈련세트,훈련세트,검증세트
#훈련세트,검증세트,훈련세트
#검증세트,훈련세트,훈련세트
#└ 패리티 방식(3-폴드 교차 검증)
#5-폴드, 10-폴드도 있음.
#사이킷런에는 cross_validate()라는 교차 검증 함수를 사용함.

from sklearn.model_selection import cross_validate
#평가할 모델 객체를 첫 번째 매개 변수로 전달함(직접 검증 세트를 떼지 않고 훈련 세트 전체를 전달함.)
scores = cross_validate(dt, train_input, train_target)
#scores = cross_validate(dt, train_input, train_target, cv=5)
#cv 매개 변수를 이용해 폴드 수를 변경함(cv=5: 기본값)
#cv = cross validation
print(scores)
#fit_time(훈련 시간), score_time(검증 시간), test_score(최종 점수) 키를 가진 딕셔너리(key:value)를 반환함.

{'fit_time': array([0.01342225, 0.01408458, 0.02848387, 0.01292944, 0.01245761]), 'score_time': array([0.00231862, 0.00571537, 0.00220442, 0.00216103, 0.00209808]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [4]:
#{'fit_time': array([0.0132041 , 0.01476693, 0.01308441, 0.01235008, 0.01488566]),
#'score_time': array([0.00205398, 0.00209689, 0.0019176 , 0.00199366, 0.00204349]),
#'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [5]:
import numpy as np

#test_score 이름이지만 검증 폴드의 점수.
#5개의 검증 점수를 평균을 내서 최종 점수를 얻음.
#0.855300214703487
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
#주의: cross_validate()는 훈련 세트를 섞어서 폴드를 나누지 않고 분할기를 사용(splitter)
#앞서 우리는 train)test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비해왔음.
#만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 함.

#사이킷런의 분할기는 교차검증에서 폴드를 어떻게 나눌지를 결정함.
#cross_validate()는 기본적으로 회귀 모델인 KFold 분할기를 사용함.
#분류 모델일 경우 타겟 클래스를 골고루 나누기 위해 StratifiedKFold를 사용함.

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
#10-폴드 교차 검증 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [9]:
#결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아봐야 함.
#테스트 세트를 사용하지 않고 교차 검증을 통해 좋은 모델을 골라야 함.

#하이퍼 파라미터 튜닝
#모델 파라미터          : 머신러닝 모델이 학습하는 파라미터
#하이퍼 파라미터        : 사용자가 지정해야만 하는 파라미터
#하이퍼 파라미터 튜닝   : 라이브러리가 제공하는 기본 값을 그대로 사용해 모델을 훈련
#그 다음 검증세트의 점수나 교차 검증을 통해 매개변수를 조금씩 변경 1~2, 5~6개의 매개변수를 제공
#AutoML                 : 사람의 개입없이 하이퍼 파라미터 튜닝을 자동으로 수행하는 기술

#Max_depth를 최적으로 고정, min_sample_split을 바꿔가며 최적의 값을 찾는다면 값이 함께 변경됨.
# → 두개의 매개변수를 동시에 바꿔가며 최적의 값을 찾아야함.
#사이킷런에서 제공하는 그리드 서치(Grid Search)가 제공.(GridSearchCV)
# └ 다중 for문은 복잡해지기 때문에 그리드 서치를 이용.

In [10]:
#그리드서치 크로스 검증용
from sklearn.model_selection import GridSearchCV
#min_impurity_decrease 매개변수의 최적값을 찾아보자.
#min_sample_split : 샘플을 최소한 몇 개 이상이어야 split(하위 노드로 분리) 할 것인지.

#클 수록 가지 치기(과대 적합 방지), 작을 수록 정확하기(과대 적합)
#0.0001 ~ 증가 5번
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

#매개변수와 탐색할 값을 리스트를 딕셔너리로 만듦.

In [14]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
#결정 트리 클래스의 객체를 생성하자마자 바로 전달함.
#일반 모델을 훈련하는 것처럼 fit() 메소드를 호출함.
#이 메소드를 호출하면 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가면서 총 5번 수행.
#GridSearchCV의 cv 매개변수 기본 값은 5. 5 * 5 = 25번 수행.
#n_jobs에서 병렬 실행에 사용될 CPU 코어수를 지정함.(-1로 사용시 모든 코어)
#CPU를 너무 많이 잡아먹기 때문에 스레드를 모두 사용해서 빠르게 훈련하도록 n_jobs를 -1을 줌.

gs.fit(train_input, train_target)
#교차 검증에서 최적의 하이퍼 파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만,
#사이킷런의 그리드 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로
#전체 훈련 세트에서 자동으로 다시 모델을 훈련함.

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [23]:
#이렇게 최적화 되어 훈련된 모델은 GridSearchCV 클래스로 만들어진
#객체의 best_estimator_ 속성에 저장되어 있다.
#최적의 매개변수는 best_params_ 속성에 저장되어 있다.

#dt.score는 0~1 사이의 실수이기 때문에 * 100
#소수점 두 번째 자리까지 나오게 하려고 :.2f
print(f'그리드 서치를 통해 찾은 최적의 모델 정확도: {dt.score(train_input, train_target)*100:.2f}%')
print(f'최적의 매개변수: {gs.best_params_}')

그리드 서치를 통해 찾은 최적의 모델 정확도: 97.06%
최적의 매개변수: {'min_impurity_decrease': 0.0001}


In [25]:
#각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장되어 있다.

#5번의 교차 검증으로 얻은 점수를 출력해보자.
#첫 번째 평균값이 제일 큰 것 같다.
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [30]:
#눈으로 보는 것보단 넘파이를 이용하여 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
#그 다음 이 인덱스를 이용해 params 키에 저장된 매개 변수를 출력해보자.
#이 값이 최상의 검증 점수를 만든 매개 변수의 조합이 된다.

#gs.cv_results_['mean_test_score'] = 0.86819297
#argmax(arguments of maxima): 최대값의 인수
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

#과정 요약
#1. 탐색할 매개 변수 지정.
#2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개 변수 조합을 찾음.
#→ 그리드 서치에 저장.
#3. 그리드 서치는 최상의 매개 변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련.
#→ 그리드 서치에 저장.

{'min_impurity_decrease': 0.0001}


In [39]:
#조금 더 복잡한 매개 변수 조합을 사용해보겠다.
#노드를 분할하기 위한 불순도 감소 최소량 지정. max_depth(트리의 깊이)
#min_sample_split 노드를 나누기 위한 최소 샘플 수.
#넘파이 arange() 함수는 첫 번째 매개 변수 값에서 시작해
#두 번째 매개 변수에 도달할 때까지 세 번째 매개 변수를 계속 더한 배열을 만듦.

#0.0001 ~ 0.001까지 0.0001을 더한 배열.(두 번째 매개 변수는 포함되지 않음) = 9번
#파이썬 arange() 함수는 정수만 사용 가능. 5~ 20까지 1씩 증가 = 15개
#교차 검증 회수 9 * 15 * 10 = 1350개 * 5-폴드 교차 = 6750개
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),#9번
          'max_depth': range(5, 20, 1),                             #5~20까지 15번
          'min_samples_split': range(2, 100, 10)                    #2~100까지 10번
         }
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [40]:
#criterion                  : 분할 품질을 측정하는 기능.(default: gini)
#splitter                   : 각 노드에서 분할을 선택하는데 사용되는 전략.(default: best)
#max_depth                  : 트리의 최대 깊이.(값이 클 수록 모델의 복잡도가 올라간다.)
#min_samples_split          : 자식 노드를 분할하는데 필요한 최소 샘플 수.(default: 2)
#min_samples_leaf           : 리프 노드에 있어야 할 최소 샘플 수.(default: 1)
#min_weight_fraction_leaf   : 'min_samples_leaf'와 같지만 가중치가 부여된 샘플 수에서의 비율.
#max_features               : 각 노드에서 분할에 사용할 특징의 최대 수.
#random_state               : 난수 seed 설정.
#max_leaf_nodes             : 리프 노드의 최대수.
#min_impurity_decrease      : 최소 불순도.
#min_impurity_split         : 나무 성장을 멈추기 위한 임계치.
#class_weight               : 클래스 가중치.
#presor                     : 데이터 정렬 필요 여부.

In [41]:
#최상의 매개 변수 조합 확인
print(gs.best_params_)
#{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [43]:
#최상의 교차 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))
#개선점: 탐색할 매개변수 간격을 0.0001 혹은 1로 설정했는데 근거 부족(좁히거나 넓힐 필요성 有)

0.8683865773302731


In [44]:
#매개 변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음.
#너무 많은 매개 변수 조건이 있어 그리드 서치 수행시간이 오래 걸림.
#해결: 랜덤 서치를 사용해보자.

#랜덤 서치: 매개 변수의 값의 목록을 전달하는 것이 아니라 매개 변수를 샘플링 할 수 있는 확률 분포도 객체를 전달.
#싸이파이: 파이썬의 핵심 과학 라이브러리로 적분, 보간, 선형대수, 확률 등 포함한 수치 계산용 전용 라이브러리(코랩에 내장되어 있음.)

from scipy.stats import uniform, randint
#uniform, randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다.*균등 분포에서 샘플링
#randint() 에서는 정수값을 추출, uniform()은 실수값을 뽑음.

In [45]:
#10개의 숫자를 샘플링(컴퓨터마다 다름)
rgen = randint(0, 10)
rgen.rvs(10)
#array([5, 8, 8, 5, 4, 2, 3, 8, 9, 8])

array([5, 8, 8, 5, 4, 2, 3, 8, 9, 8])

In [51]:
#1000개를 샘플링 해서 각 숫자의 개수를 세어봄.
np.unique(rgen.rvs(1000), return_counts=True)
# (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
#  array([105, 112, 104,  90,  99,  91,  92, 100,  97, 110]))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 112, 104,  90,  99,  91,  92, 100,  97, 110]))

In [52]:
#실수 테스트
ugen = uniform(0,1)
ugen.rvs(10)

#난수 발생기(랜덤과 유사)

array([0.87727458, 0.04972099, 0.58751388, 0.7518514 , 0.74983374,
       0.66510315, 0.92469598, 0.68291031, 0.46645968, 0.16621521])

In [54]:
#탐색할 매개변수의 딕서너리를 만들어 봄.
#min_samples_leaf를 탐색 대상에 추가.
params = {'min_impurity_decrease': uniform(0.0001, 0.001),#0.0001 ~ 0.001의 실수값
          'max_depth': randint(20, 50),#20 ~ 50 사이 정수
          'min_samples_split': randint(2, 25),#2 ~ 25 사이 정수
          'min_samples_leaf': randint(1, 25)#1 ~ 25 사이 정수
          #리프 노드가 되기 위한 최소 샘플 개수(지식 노드의 샘플 수가 이 값보다 작으면)
        }

In [55]:
#샘플링 회수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개 변수에 지정
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
#n_iter=100 총 100번을 샘플링 하여 교차 검증을 수행(최적의 매개 변수 조합을 찾음.)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab0ab368a90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ab0ab36a8c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab0abdcfa00>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab0ab369fc0>},
                   random_state=42)

In [60]:
#최적의 매개변수 조합을 찾음.
print(gs.best_params_)
#{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [61]:
#최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [63]:
#최종 모델로 결정하고 테스트 세트의 성능을 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

#검증점수: 86.95
#테스트  : 86

0.86
